# Connect to historic data API

In [1]:
# Free API, no credentials needed
import yfinance as yf

import pandas as pd
import numpy as np

import utils
import requests
from datapackage import Package, Resource

/Users/come.vitis/Documents/Personnal/Projects/finance-env/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Constants
TICKERS = {
    "S&P500": "^GSPC"
}

In [3]:
# Load heavy resources here

# Get the constituents of S&P500 
sp500_constituents = utils.load_sp_500_constituents() # Default scrap mode is false

In [4]:
# Arbitrarly for now: select 5 companies
top_5 = sp500_constituents[:5]

In [5]:
# Get the quaterly report release dates of those tickers
top_5_symbols = [row.symbol for _, row in top_5.iterrows()]
quaterly_releases = utils.load_ticker_earnings_history(top_5_symbols, reload=False)

In [6]:
quaterly_releases

,symbol,company,earning_dates,eps_estimates,eps_reported,surprise_percent
index,,,,,,
0,AOS,A. O. Smith Corporation,"Jul 28, 2022, 2 AMEDT",0.81,0.82,1.61
1,AOS,A. O. Smith Corporation,"Apr 28, 2022, 2 AMEDT",0.75,0.77,3.08
2,AOS,A. O. Smith Corporation,"Jan 27, 2022, 1 AMEST",0.77,0.87,13.28
3,AOS,A. O. Smith Corporation,"Oct 28, 2021, 2 AMEDT",0.68,0.82,20.59
4,AOS,A. O. Smith Corporation,"Jul 29, 2021, 2 AMEDT",0.65,0.73,12.31
...,...,...,...,...,...,...
429,MMM,3M Company,"Oct 22, 1998, 12 AMEDT",0.49,0.49,-1.11
430,MMM,3M Company,"Jul 23, 1998, 12 AMEDT",0.46,0.48,3.60
431,MMM,3M Company,"Apr 28, 1998, 12 AMEDT",0.50,0.49,-1.37
